In [1]:
import math
import torch

from matplotlib import pyplot as plt
%matplotlib inline

import os
import argparse
import pandas as pd
from pandas import factorize
import numpy as np
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
#from sklearn.model_selection import KFold

import gpytorch
import itertools


In [2]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
"""
import m3hgpr
from m3hgpr.config import get_cfg_defaults_all
from m3hgpr.load_data import Data_set
from m3hgpr.models import singleGP_gpytorch,rf_sklearn,singleGP_gpytorch_train,singleGP_gpytorch_reference,MultitaskGPModel,MTMOGP
from m3hgpr.train import CV_Trainer
"""

In [10]:
from m3gpr.config import get_cfg_defaults_all
from m3gpr.load_data import Data_set
from m3gpr.models import singleGP_gpytorch,rf_sklearn,singleGP_gpytorch_train,singleGP_gpytorch_reference,MultitaskGPModel,MTMOGP
from m3gpr.train import CV_Trainer

In [11]:
cfg = get_cfg_defaults_all()
os.chdir('/Users/chenya68/Documents/GitHub/BFO')
parser = argparse.ArgumentParser(description="Configure files")
parser.add_argument("--cfg", required=True, help="path to config file", type=str)
args = parser.parse_args(['--cfg', 'data/harpoon/harpoon_dataset.yaml'])
cfg.merge_from_file(args.cfg)

cfg.freeze()
setup_data = Data_set(cfg, 1)
print(cfg.MODEL.MODEL_NAME)
print(cfg.MODEL.Y_SCALE)

#df_x, df_y_wide, df_y_long = setup_data.Load_data_set()
df_x, df_y,ls_X_train,ls_X_test,ls_y_train,ls_y_test, obj_y_scaler= setup_data.Load_data_set()
print(df_x.shape)
print(df_y.shape)
#print(df_y_wide.shape)
#print(df_y_long.shape)

Creating folder /Users/chenya68/Documents/GitHub/BFO/output/Harpoon/MTMO/by-task/djan2026/num_tasks_2
MTMO
y-stand
ohe
(48, 10)
(48, 2)


## Single GP Cross-Validation

In [5]:
if True:
    sgp_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = sgp_cv_trainer.set_up_sgp_cv()
    
    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    
    #num_tasks = cfg.DATA.NUM_TASKS
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
    #for j in range(num_total_outputs):
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    #output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE])
    df_test_metrics.to_csv(setup_data.output_path+'/df_test_metrics.csv')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
number of tasks that are being compared:  6
best_idx 16
best combinations:  (4.0, 0.1, 200)
best mean mae 0.494
best mean r2 0.484
best mean err 0.919
best mae of target 0 is 0.493
best r2 of target 0 is 0.668
best err of target 0 is 0.955
best mae of target 1 is 0.679
best r2 of target 1 is 0.384
best err of target 1 is 1.163
best mae of target 2 is 0.216
best r2 of target 2 is 0.415
best err of target 2 is 0.382
best mae of target 3 is 0.307
best r2 of target 3 is 0.214
best err of target 3 is 0.540
best mae of target 4 is 0.447
best r2 of target 4 is 0.771
best err of target 4 is 0.924
best mae of target 5 is 0.822
best r2 of target 5 is 0.452
best err of target 5 is 1.549
         total  Task_1_Output_1  Task_1_Output_2  Task_2_Output_1

## MO Cross-Validation

In [5]:
mo_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mo_cv_trainer.set_up_cv_model()

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [34]:
print('both targets')
best_idx = np.argmin(mean_arr_cv_mae[1,:])

print('best_idx',best_idx)
print('mean min mae %.3f' %(np.min(mean_arr_cv_mae[1,:])))

best_idx = np.argmax(mean_arr_cv_r2[1,:])
#print('best_idx',best_idx)
print('mean max r2 %.3f' %(np.max(mean_arr_cv_r2[1,:])))
print('mean min err %.3f' %(mean_arr_cv_err[1,best_idx]))

for j in range(cfg.DATA.NUM_OUTPUTS):
    print('min mae of target %d is %.3f' %(j, ls_arr_cv_mae[j][1,best_idx]))
#print('min mae target 1',ls_arr_cv_mae[1][1,best_idx])

    print('max r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
#print('max r2 target 1',arr_cv_r2_1[1,best_idx])

    print('min err of target %d is %.3f' %(j,ls_arr_cv_err[j][1,best_idx]))
#print('min err target 1',arr_cv_err_1[1,best_idx])

print(combs[best_idx])


both targets
best_idx 4
mean min mae 0.430
mean max r2 0.462
mean min err 1.343
min mae of target 0 is 0.436
max r2 of target 0 is 0.625
min err of target 0 is 1.387
min mae of target 1 is 0.549
max r2 of target 1 is 0.352
min err of target 1 is 1.766
(0.35, 700)


In [6]:
if True:
    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    
    #num_tasks = cfg.DATA.NUM_TASKS
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
        
    cols_target_new = ['total']
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
    #for j in range(num_total_outputs):
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    df_test_metrics.to_csv(setup_data.output_path+'/df_test_metrics.csv')

number of tasks that are being compared:  6
best_idx 2
best combinations:  (0.25, 900)
best mean mae 0.599
best mean r2 0.335
best mean err 1.346
best mae of target 0 is 0.658
best r2 of target 0 is -0.443
best err of target 0 is 1.300
best mae of target 1 is 0.740
best r2 of target 1 is 0.056
best err of target 1 is 1.609
best mae of target 2 is 0.255
best r2 of target 2 is 0.367
best err of target 2 is 0.588
best mae of target 3 is 0.352
best r2 of target 3 is 0.298
best err of target 3 is 0.860
best mae of target 4 is 0.601
best r2 of target 4 is 0.545
best err of target 4 is 1.165
best mae of target 5 is 0.988
best r2 of target 5 is 0.442
best err of target 5 is 2.553
         total  Task_1_Output_1  Task_1_Output_2  Task_2_Output_1  \
RMSE     0.599            0.658            0.740            0.255   
R2       0.335           -0.443            0.056            0.367   
AVG_STD  1.346            1.300            1.609            0.588   

         Task_2_Output_2  Task_3_Output_1 

## MTMO Cross validation

In [12]:
if True:
    mtmo_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mtmo_cv_trainer.set_up_cv_model()

    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    #for j in range(cfg.DATA.NUM_OUTPUTS):
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = [cfg.MODEL.METRIC,'R2','AVG_STD'])
    print(df_test_metrics)
    
    output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE,'num_tasks_'+str(cfg.DATA.NUM_TASKS)])
    df_test_metrics.to_csv(output_path+'/df_test_metrics.csv')

number of tasks that are being compared:  4
best_idx 7
best combinations:  (0.15, 1100)
best mean mae 0.502
best mean r2 0.578
best mean err 0.879
best mae of target 0 is 0.363
best r2 of target 0 is 0.616
best err of target 0 is 0.612
best mae of target 1 is 0.517
best r2 of target 1 is 0.483
best err of target 1 is 1.142
best mae of target 2 is 0.368
best r2 of target 2 is 0.789
best err of target 2 is 0.609
best mae of target 3 is 0.759
best r2 of target 3 is 0.422
best err of target 3 is 1.151
         total  Task_1_Output_1  Task_1_Output_2  Task_2_Output_1  \
RMSE     0.502            0.363            0.517            0.368   
R2       0.578            0.616            0.483            0.789   
AVG_STD  0.879            0.612            1.142            0.609   

         Task_2_Output_2  
RMSE               0.759  
R2                 0.422  
AVG_STD            1.151  


## MTMO-LMGP CV

In [5]:
if True:
    mtmolmgp_cv_trainer = CV_Trainer(cfg,ls_X_train,ls_X_test,ls_y_train,ls_y_test,obj_y_scaler)
    combs,ls_model_from_combs,ls_arr_cv_mae,ls_arr_cv_r2,ls_arr_cv_err,mean_arr_cv_mae,mean_arr_cv_r2,mean_arr_cv_err = mtmolmgp_cv_trainer.set_up_cv_model()

    if cfg.MODEL.SPLIT == 'by-task':
        num_tasks = cfg.DATA.NUM_TASKS
    else:
        num_tasks = 1
    num_outputs = cfg.DATA.NUM_OUTPUTS
    num_total_outputs = num_outputs*num_tasks
    arr_metrics = np.zeros((3,1+num_total_outputs)) #row:mae, r2, avg_std; col: total mean, task1(output1), task2(output2),...

    print('number of tasks that are being compared: ',num_total_outputs)

    best_idx = np.argmin(mean_arr_cv_mae[1,:])
    #best_idx = np.argmax(mean_arr_cv_r2[1,:])

    print('best_idx',best_idx)
    print('best combinations: ',combs[best_idx])

    arr_metrics[0,0] = np.min(mean_arr_cv_mae[1,:])
    arr_metrics[1,0] = np.max(mean_arr_cv_r2[1,:])
    arr_metrics[2,0] = mean_arr_cv_err[1,best_idx]

    print('best mean mae %.3f' % arr_metrics[0,0])
    print('best mean r2 %.3f' % arr_metrics[1,0])
    print('best mean err %.3f' % arr_metrics[2,0])
    
    cols_target_new = ['total']
    #for j in range(cfg.DATA.NUM_OUTPUTS):
    for task_ind in range(num_tasks):
        for out_ind in range(num_outputs):
            j = out_ind + num_outputs*task_ind
            j_name = 'Task_'+str(task_ind+1) + '_Output_'+str(out_ind+1)
            cols_target_new.append(j_name)
            arr_metrics[0,j+1] = ls_arr_cv_mae[j][1,best_idx]
            arr_metrics[1,j+1] = ls_arr_cv_r2[j][1,best_idx]
            arr_metrics[2,j+1] = ls_arr_cv_err[j][1,best_idx]
            print('best mae of target %d is %.3f' %(j,ls_arr_cv_mae[j][1,best_idx]))
            print('best r2 of target %d is %.3f' %(j,ls_arr_cv_r2[j][1,best_idx]))
            print('best err of target %d is %.3f' % (j,ls_arr_cv_err[j][1,best_idx]))
    
    df_test_metrics = pd.DataFrame(np.round(arr_metrics,3),columns = cols_target_new, 
                               index = ['MAE','R2','AVG_STD'])
    print(df_test_metrics)
    
    output_path = '/'.join([cfg.PATH.RESULT,cfg.MISC.DATA_NAME,cfg.MODEL.MODEL_NAME,cfg.MODEL.SPLIT,cfg.MISC.DATE,'num_tasks_'+str(cfg.DATA.NUM_TASKS)])
    df_test_metrics.to_csv(output_path+'/df_test_metrics.csv')

## Learning the model's parameters has started ##


Epoch:   0%|          | 0/100 [00:00<?, ?it/s]

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Epoch 99 - loss 73.5903: 100%|██████████| 100/100 [00:00<00:00, 114.78it/s]


## Learning the model's parameters is successfully finished ##
## Learning the model's parameters has started ##


...

## Learning the model's parameters is successfully finished ##
## Learning the model's parameters has started ##


Epoch 99 - loss 50.5956: 100%|██████████| 100/100 [00:05<00:00, 17.72it/s]


## Learning the model's parameters is successfully finished ##
## Learning the model's parameters has started ##


Epoch 99 - loss 48.8303: 100%|██████████| 100/100 [00:06<00:00, 15.20it/s]


## Learning the model's parameters is successfully finished ##
number of tasks that are being compared:  6
best_idx 1
best combinations:  (0.02, 100)
best mean mae 0.574
best mean r2 -2.198
best mean err 1.334
best mae of target 0 is 0.634
best r2 of target 0 is -3.551
best err of target 0 is 1.281
best mae of target 1 is 0.736
best r2 of target 1 is -5.201
best err of target 1 is 1.440
best mae of target 2 is 0.286
best r2 of target 2 is -0.716
best err of target 2 is 1.195
best mae of target 3 is 0.352
best r2 of target 3 is -0.255
best err of target 3 is 1.328
best mae of target 4 is 0.666
best r2 of target 4 is -0.858
best err of target 4 is 1.302
best mae of target 5 is 0.772
best r2 of target 5 is -2.608
best err of target 5 is 1.461
         total  Task_1_Output_1  Task_1_Output_2  Task_2_Output_1  \
MAE      0.574            0.634            0.736            0.286   
R2      -2.198           -3.551           -5.201           -0.716   
AVG_STD  1.334            1.281            